In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import normaltest, norm
import time
%matplotlib inline

In [117]:
processed_dir= "C:/Users/Hermans_Desktop1/OneDrive - UW-Madison/Documents/Research/2022-P4 (Pt DH ML)/iterative_catalyst_design/data/processed"
models_dir= "C:/Users/Hermans_Desktop1/OneDrive - UW-Madison/Documents/Research/2022-P4 (Pt DH ML)/iterative_catalyst_design/models"

#processed_dir = "D:/ukuru/OneDrive - UW-Madison/Documents/Research/2022-P4 (Pt DH ML)/iterative_catalyst_design/data/processed"
infile = processed_dir+"/"+"initial_catalyst_data_all.csv"
outfile_stripped = processed_dir+"/"+"initial_catalyst_data_stripped.csv"
outfile_avg = processed_dir+"/"+"initial_catalyst_data_averaged.csv"
outfile_pred = models_dir+"/"+"test_prediction.csv"

# 1) Perform cross validation

Produces a matrix where all entries of the same metal composition are averaged.

# 3) Developing Grid of Possible Catalysts

Develops a grid of all possible catalytic materials to perform a performance prediction on.

There are 5 metals varied in this screen (Sn, Ga, Fe, Cu, Ca). For each metal, any ratio to Pt of 0-8 with 0.5 increments is possible, leading to $17^5=1419857$ possible materials for screening.

In [120]:
X_grid = np.mgrid[0:8:17j, 0:8:17j,0:8:17j,0:8:17j,0:8:17j].reshape(5,-1).T


X_grid.shape

(1419857, 5)

# 4) Perform KNN regression for every catalyst
Performs KNN regression for every catalyst in our grid. Uses bootstrap of experimental data with standard deviations included in bootstrap sampling to produce 'exemplar' ML data. 

In [133]:
metals = ["Sn","Ga","Fe","Cu","Ca"]
cols = metals.copy()
cols.append("lifetime_yield")
cols.append("lifetime_yield_sd")

expt_data = df_avg[cols].to_numpy()
X_expt = expt_data[:,:-2]
y_expt = expt_data[:,-2:]
num_datapts = len(expt_data) #number of distinct inputs
seed = 1



#define train, val, test splits. See https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train,X_test, y_train, y_test = train_test_split(X_expt,y_expt,test_size=.2,random_state=seed)
X_train,X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=.25,random_state=seed) #.25 * .8 = .2 overall fraxn for val
for label,arr in zip(["Train","Validation","Test"],[y_train, y_val, y_test]):
    print(f'{len(arr)} data points in {label} set.')


def bootstrap(X,y,n=1000,seed=1):
    """Produces sampled experimental data of n points. It does this by randomly sampling existing data with repeats possible. 
    Next, it assumes lifetime yield is normally distributed and uses the AVG and SD values for a given experimental datapoint
    to estimate the lifetime yield of a given datapoint."""
    rng = np.random.default_rng(seed=seed)
    datapts = rng.integers(low=0,high=len(X),size=n) #randomly get n sample indices
    lifetime_yield = rng.normal(loc=y[datapts][:,0],scale=y[datapts][:,1]) #calculate lifetime yield by sampling normal distribution for each catalyst
    return X[datapts],lifetime_yield #return the metal loadings and lifetime yield estimate

15 data points in Train set.
6 data points in Validation set.
6 data points in Test set.


In [158]:
def KNN_regressor(X_train,y_train,X_test,n_bootstrap=1000,n_split=20,weights='uniform'):
    """
    Implements the KNN Regressor with selected options of weight. Bootstraps the training data (default=1000 samples)
    and splits it into a chosen number of arrays. Returns the average and std dev of the predictions.
    Default weighting of KNN Regressor is 'uniform', but 'distance' is also possible
    Return (mu {length = len(X_test)}, sigma {length = len(X_test)})"""
    t0 = time.time()
    X,y = bootstrap(X_train,y_train,n=n_bootstrap)
    Xs = np.array_split(X,n_split)
    ys = np.array_split(y,n_split)
    predictions = np.asarray([KNeighborsRegressor(weights=weights).fit(Xs[i],ys[i]).predict(X_test) for i in range(n_split)])
#     print(normaltest(predictions,axis=0))
    mu = np.average(predictions,axis=0)
    sigma = np.std(predictions,axis=0)
#     return np.vstack((mu,sigma))
    t1 = time.time()
    print(f'KNN Run Time {t1-t0:.5} seconds')
    return mu,sigma

def EI(X_train,y_train,X_test,n_bootstrap=1000,n_split=10,pred_type = 'KNN_uniform'):
    """Returns the expected improvement function for all points in X_test. Implements KNN.
    Return: (explore, exploit) where both are arrays of length len(X_test)"""
    mu_star = np.max(y_train)
    if pred_type == 'KNN_uniform':
        mu, sigma = KNN_regressor(X_train,y_train,X_test,n_split=n_split,n_bootstrap=n_bootstrap,weights='uniform')
    elif pred_type == 'KNN_distance':
        mu, sigma = KNN_regressor(X_train,y_train,X_test,n_split=n_split,n_bootstrap=n_bootstrap,weights='distance')
    Z = (mu-mu_star)/sigma #element-wise
    explore = sigma*norm.pdf(Z)
    exploit = norm.cdf(Z)*sigma*Z #element-wise
    
    return (explore,exploit,mu,sigma) 



In [147]:
X_val
metals

['Sn', 'Ga', 'Fe', 'Cu', 'Ca']

In [159]:
#testing on full experimental data, no hyperparameter tuning
t0 = time.time()
explore,exploit,mu,sigma = EI(X_expt,y_expt,X_grid)
t1 = time.time()
print(f'EI Run Time {t1-t0:.5} seconds')
df_pred = pd.DataFrame(X_grid,columns=metals)
df_pred["Explore"] = explore
df_pred["Exploit"] = exploit
df_pred["EI_Score"] = df_pred["Explore"] + df_pred["Exploit"]
df_pred["mu"] = mu
df_pred["sigma"] = sigma
df_pred.sort_values(by=["EI_Score"],ascending=False,inplace=True)
t2 = time.time()
print(f'Pandas Run Time {t2-t1:.5} seconds')




KNN Run Time 36.541 seconds
EI Run Time 36.885 seconds
Pandas Run Time 0.53167 seconds


In [160]:
df_pred.head(n=20)

,Sn,Ga,Fe,Cu,Ca,Explore,Exploit,EI_Score,mu,sigma
610667,3.5,2.5,2.5,0.0,5.0,32.517994,-17.120075,15.397919,543.720055,101.729427
610670,3.5,2.5,2.5,0.0,6.5,32.258727,-17.139145,15.119581,542.761135,101.611761
527166,3.0,2.5,2.5,0.5,6.5,32.258727,-17.139145,15.119581,542.761135,101.611761
610668,3.5,2.5,2.5,0.0,5.5,32.258727,-17.139145,15.119581,542.761135,101.611761
527165,3.0,2.5,2.5,0.5,6.0,32.258727,-17.139145,15.119581,542.761135,101.611761
610669,3.5,2.5,2.5,0.0,6.0,32.258727,-17.139145,15.119581,542.761135,101.611761
527164,3.0,2.5,2.5,0.5,5.5,32.258727,-17.139145,15.119581,542.761135,101.611761
527163,3.0,2.5,2.5,0.5,5.0,32.258727,-17.139145,15.119581,542.761135,101.611761
610954,3.5,2.5,3.0,0.0,4.0,30.934319,-15.822049,15.112270,551.231927,94.849376
591319,3.5,0.5,3.0,0.5,4.0,30.934319,-15.822049,15.112270,551.231927,94.849376
